$$\textbf{Optimización I. Tarea 7}$$
$$\textit{Y. Sarahi García Gozález}$$

$\color{purple}{Librerías \space }$ <br>

In [109]:
import numpy as np
import matplotlib.pyplot as plt
import functions as fn #libreria con las funciones Himmelblau,Beale,Rosenbrock,Hartman
from math import exp
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad  

In [86]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version", python_version())
print("Numpy version", np.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version 3.11.7
Numpy version 1.26.3


In [87]:
#imprimimos el epsilon de la máquina
epsilon = np.finfo(float).eps
print("Epsilon de la máquina:", epsilon)

Epsilon de la máquina: 2.220446049250313e-16


In [88]:
#definicion de funcion para para las graficas dee los contornos

def contornosFnc2D(fncf, puntos, xleft, xright, ybottom, ytop, levels):
    # Crea una discretización uniforme del intervalo [xleft, xright]
    ax = np.linspace(xleft, xright, 250)
    # Crea una discretización uniforme del intervalo [ybottom, ytop]
    ay = np.linspace(ybottom, ytop, 200)
    # La matriz mX que tiene las abscisas 
    mX, mY = np.meshgrid(ax, ay)
    # Se crea el arreglo mZ con los valores de la función en cada nodo
    mZ = mX.copy()
    for i, y in enumerate(ay):
        for j, x in enumerate(ax):
            mZ[i,j] = fncf(np.array([x, y]))
    # Grafica de las curvas de nivel
    fig, ax = plt.subplots()
    CS = ax.contourf(mX, mY, mZ, levels, cmap='coolwarm')
    plt.colorbar(CS, ax=ax)
    # Grafica los puntos dados
    puntos_x = [p[0] for p in puntos]
    puntos_y = [p[1] for p in puntos]
    ax.plot(puntos_x, puntos_y, 'r-',label="Sucesión")
    ax.plot(puntos_x[0], puntos_y[0], 'g*',label="punto inicial")
    ax.plot(puntos_x[-1], puntos_y[-1], 'b*',label="mínimo encontrado")
     # Grafica los puntos como puntos rojos
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title('Contornos de la función sucesión')
    plt.legend()
    plt.show()

In [89]:
def imprime(f,n,x0,metodo,metodo_args):

    '''
    Esta función imprime:
    - la dimensión $n$,
    - f(x_0)
    - el  número $k$ de iteraciones realizadas
    - f(x_k)
    - las primeras y últimas 4 entradas del punto $\mathbf{x}_k$ que devuelve el algoritmo,
    - la norma del gradiente g_k, 
    - promedio de iteraciones del algoritmo 1
    - la variable $bres$ para saber si el algoritmo puedo converger.

    '''

    xk,fk,gk,k,iteraciones,indicador= metodo(*metodo_args)

    prom=np.mean(iteraciones)

    print('Dimensión n = ', n )
    print('f(x_0) = ', f(x0) )
    print('Número de iteraciones = ', k )
    print('f(x_k) = ', fk )
    print('Primeras cuatro entradas de x_k= ', xk[:4])
    print('Últimas cuatro entradas de x_k= ', xk[-4:])
    print('Norma del gradiente ||gk||   = ', np.linalg.norm(gk))
    print('Iteraciones promedio  = ', prom)


    if(indicador):
        print("Sí se cumplio el criterio de convergencia")


In [90]:
#definimos la funcion que genera la matriz A de acuerdo a la instrucción anterior
def genera_A1(n):
    return np.ones((n,n))+n*np.eye(n)
#definimos la funcion que genera la matriz A de acuerdo a la instrucción anterior
def genera_A2(n):
    A = np.empty([n,n], dtype=float)
    for i in range(n):
        for j in range(n):
            u=0.25*((i-j)**2)
            A[i][j]=exp(-1*u)
    return A

$\color{purple}{Ejercicio \space 1}$ <br>

In [91]:
def grad_des_newton(Df_k,Hf_k,n,tol):
    iteraciones=0

    zk=0
    rk=Df_k
    dk=-rk
    
    for k in range(n):
        m=np.dot(dk,Hf_k@dk)
        if(m)<=0:
            if k==0:
                return dk, k
            else:
                return zk, k
        
        r=np.linalg.norm(rk) #norma del gradiente


        alpha_k=(r**2)/m
        z_old=zk
        zk=zk+alpha_k*dk #actualiamos z
        rk=rk+alpha_k*((Hf_k@dk)) #actualizamos r

        r_new=np.linalg.norm(rk) #norma del gradiente actualizado
        beta_k=((r_new)**2)/(r**2)
        dk=-rk+beta_k*dk

        if r_new<tol:
            return zk,k
        
        iteraciones=iteraciones+1

    return z_old, iteraciones

def backtracking(alpha_ini,x_k,f,f_k,df,p_k,rho=0.5,c=0.001,iter_max=500):

    '''
        Esta funcion parte de un tamaño de paso inicial alpha_ini y lo va recortando hasta que
        cumple la cond de descenso suficiente

        parametros:
            valores (float): alpha_ini, rho entre (0,1), f(x_k), Df(x_k) (gradiente en el punto x_k), c_1, 
            direccion de descenso (np.rray): p_k 

        returns:
            el tamaño de paso a_k
            numero de iteraciones realizadas i_k
    '''

    alpha=alpha_ini #fijamos alpha como el alpha inicial
     
    for i in range(iter_max):
        x_kp=x_k+alpha*p_k
        gp=c*np.dot(df(x_k),p_k) #hacemos el producto gradiente por direccion de descenso p
    

        #si la condicion de descenso se cumple, terminamos
        if f(x_kp)<=(f_k + alpha*gp):
            return alpha

        alpha=alpha*rho #si no se cumple la cond, hacemos alpha*rho
     
    return  alpha




In [92]:
def Newton_truncado(f,Df,Hf,x0,tol,max_iter=5000):
    iteraciones=[]
    xk=x0
    
    for k in range(max_iter):
        #inicializamos xk y el gradiente en xk  
        gk=Df(xk)

        #norma del gradiente
        g=np.linalg.norm(gk)
        if g<tol: #condicion de paro
            return xk,f(xk),gk,k,iteraciones,True

        Hk=Hf(xk) #hessiana

        #calculamos la tolerancia para el algortimo 1
        tolerancia=min(0.5,np.sqrt(g))*g 

        #llamamos al algoritmo 1 que regresa pk y el num me iteraciones i
        pk,i=grad_des_newton(gk,Hk,len(x0),tolerancia)   
        iteraciones.append(i) #agregamos el num de iteraciones a una lista

        #calculamos alpha
        alpha_k=backtracking(1,xk,f,f(xk),Df,pk)
        #calculamos el siguiente xk
        xk=xk+alpha_k*pk

    return xk,f(xk),gk,k,iteraciones,False

**Función cuadrática con A1**

$n=10$

In [93]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica, su gradiente y su Hessiana como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
H_cuadratica = lambda x: A

argumentos_NT=[cuadratica,D_cuadratica,H_cuadratica,x0,tau]
imprime(cuadratica,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  10
f(x_0) =  0.0
Número de iteraciones =  1
f(x_k) =  -0.25
Primeras cuatro entradas de x_k=  [0.05 0.05 0.05 0.05]
Últimas cuatro entradas de x_k=  [0.05 0.05 0.05 0.05]
Norma del gradiente ||gk||   =  1.041481514324134e-15
Iteraciones promedio  =  0.0
Sí se cumplio el criterio de convergencia


$n=100$

In [94]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica, su gradiente y su Hessiana como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
H_cuadratica = lambda x: A

argumentos_NT=[cuadratica,D_cuadratica,H_cuadratica,x0,tau]
imprime(cuadratica,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  100
f(x_0) =  0.0
Número de iteraciones =  1
f(x_k) =  -0.2500000000000001
Primeras cuatro entradas de x_k=  [0.005 0.005 0.005 0.005]
Últimas cuatro entradas de x_k=  [0.005 0.005 0.005 0.005]
Norma del gradiente ||gk||   =  8.005932084973442e-16
Iteraciones promedio  =  0.0
Sí se cumplio el criterio de convergencia


$n=1000$

In [95]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A1(n)
b=np.ones(n)
x0=np.zeros(n)
#definimos la funcion cuadratica, su gradiente y su Hessiana como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
H_cuadratica = lambda x: A

argumentos_NT=[cuadratica,D_cuadratica,H_cuadratica,x0,tau]
imprime(cuadratica,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  1000
f(x_0) =  0.0
Número de iteraciones =  1
f(x_k) =  -0.25000000000000033
Primeras cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Últimas cuatro entradas de x_k=  [0.0005 0.0005 0.0005 0.0005]
Norma del gradiente ||gk||   =  5.600639195426344e-13
Iteraciones promedio  =  0.0
Sí se cumplio el criterio de convergencia


**Función cuadrática con A2**

$n=10$

In [96]:
n=10
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)

argumentos_NT=[cuadratica,D_cuadratica,H_cuadratica,x0,tau]
imprime(cuadratica,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  10
f(x_0) =  0.0
Número de iteraciones =  4
f(x_k) =  -1.7934208025263514
Primeras cuatro entradas de x_k=  [ 1.36909916 -1.16637682  1.60908281 -0.61339053]
Últimas cuatro entradas de x_k=  [-0.61339053  1.60908281 -1.16637682  1.36909916]
Norma del gradiente ||gk||   =  3.379705256316069e-13
Iteraciones promedio  =  2.25
Sí se cumplio el criterio de convergencia


$n=100$

In [97]:
n=100
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)

#definimos la funcion cuadratica, su gradiente y su Hessiana como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
H_cuadratica = lambda x: A

argumentos_NT=[cuadratica,D_cuadratica,H_cuadratica,x0,tau]
imprime(cuadratica,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  100
f(x_0) =  0.0
Número de iteraciones =  7
f(x_k) =  -14.494331329457918
Primeras cuatro entradas de x_k=  [ 1.446246   -1.41622759  2.11026863 -1.42471103]
Últimas cuatro entradas de x_k=  [-1.42471952  2.11026687 -1.41621544  1.44623942]
Norma del gradiente ||gk||   =  1.883728586011044e-05
Iteraciones promedio  =  32.285714285714285
Sí se cumplio el criterio de convergencia


$n=1000$

In [98]:
n=1000
tau=(np.sqrt(n))*((epsilon)**(1/3))
A=genera_A2(n)
b=np.ones(n)
x0=np.zeros(n)

#definimos la funcion cuadratica, su gradiente y su Hessiana como inline functions
cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
D_cuadratica = lambda x: 0.5 * (np.dot(A.T, x) + np.dot(A, x)) - b
H_cuadratica = lambda x: A

argumentos_NT=[cuadratica,D_cuadratica,H_cuadratica,x0,tau]
imprime(cuadratica,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  1000
f(x_0) =  0.0
Número de iteraciones =  7
f(x_k) =  -141.43698763227366
Primeras cuatro entradas de x_k=  [ 1.44628334 -1.41634395  2.11049252 -1.42505022]
Últimas cuatro entradas de x_k=  [-1.42505022  2.11049252 -1.41634395  1.44628334]
Norma del gradiente ||gk||   =  8.21944409663772e-06
Iteraciones promedio  =  60.42857142857143
Sí se cumplio el criterio de convergencia


**Función de Beale**

In [99]:
x0=(2,3)
n=2
tau=(np.sqrt(n))*((epsilon)**(1/3))
argumentos_NT=[fn.Beale,fn.D_Beale, fn.H_Beale,x0,tau]
imprime(fn.Beale,n,x0,Newton_truncado,argumentos_NT)


Dimensión n =  2
f(x_0) =  3347.203125
Número de iteraciones =  77
f(x_k) =  6.3645290412639565e-12
Primeras cuatro entradas de x_k=  [2.99999402 0.49999835]
Últimas cuatro entradas de x_k=  [2.99999402 0.49999835]
Norma del gradiente ||gk||   =  7.716816163144511e-06
Iteraciones promedio  =  0.7662337662337663
Sí se cumplio el criterio de convergencia


**Función de Himmelblau** 


In [166]:
x0=(2,4)
n=len(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
argumentos_NT=[fn.Himmelblau,fn.D_Himmelblau, fn.H_Himmelblau,x0,tau]
imprime(fn.Himmelblau,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  2
f(x_0) =  130
Número de iteraciones =  7
f(x_k) =  8.829864314112677e-13
Primeras cuatro entradas de x_k=  [2.99999995 2.00000025]
Últimas cuatro entradas de x_k=  [2.99999995 2.00000025]
Norma del gradiente ||gk||   =  7.39505206237883e-06
Iteraciones promedio  =  0.42857142857142855
Sí se cumplio el criterio de convergencia


**Función de Rosenbrock**

In [101]:
x0=(-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
argumentos_NT=[fn.Rosenbrock,fn.D_Rosenbrock, fn.H_Rosenbrock,x0,tau]
imprime(fn.Rosenbrock,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  (2,)
f(x_0) =  24.199999999999996
Número de iteraciones =  63
f(x_k) =  2.8357872857637625e-15
Primeras cuatro entradas de x_k=  [1.         0.99999999]
Últimas cuatro entradas de x_k=  [1.         0.99999999]
Norma del gradiente ||gk||   =  2.3662776926737386e-06
Iteraciones promedio  =  0.7301587301587301
Sí se cumplio el criterio de convergencia


In [102]:
x0=(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
argumentos_NT=[fn.Rosenbrock,fn.D_Rosenbrock, fn.H_Rosenbrock,x0,tau]
imprime(fn.Rosenbrock,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  (20,)
f(x_0) =  4597.999999999999
Número de iteraciones =  126
f(x_k) =  2.9610787696105527e-13
Primeras cuatro entradas de x_k=  [1. 1. 1. 1.]
Últimas cuatro entradas de x_k=  [0.99999988 0.99999977 0.99999953 0.99999906]
Norma del gradiente ||gk||   =  1.316761060292807e-06
Iteraciones promedio  =  2.8492063492063493
Sí se cumplio el criterio de convergencia


In [103]:
x0=(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)
n=np.shape(x0)
tau=(np.sqrt(n))*((epsilon)**(1/3))
argumentos_NT=[fn.Rosenbrock,fn.D_Rosenbrock, fn.H_Rosenbrock,x0,tau]
imprime(fn.Rosenbrock,n,x0,Newton_truncado,argumentos_NT)

Dimensión n =  (40,)
f(x_0) =  9680.0
Número de iteraciones =  184
f(x_k) =  2.40629420102252e-14
Primeras cuatro entradas de x_k=  [1. 1. 1. 1.]
Últimas cuatro entradas de x_k=  [0.99999997 0.99999994 0.99999987 0.99999974]
Norma del gradiente ||gk||   =  1.3405342703442557e-06
Iteraciones promedio  =  3.505434782608696
Sí se cumplio el criterio de convergencia


$\color{purple}{Ejercicio \space 2}$ <br>

In [104]:
def forward_difference(f, x, h):
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        x_forward = np.copy(x)
        #sumamos h en la iesima entrada
        x_forward[i] += h 
        #calculamos la iesima derivada parcial
        grad[i] = (f(x_forward) - f(x)) / h 
    return grad


def forward_difference_hessian(f, x, h):
    n = len(x)
    hessian = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i <= j:
                #vector x + he_i + he_j
                x_ij = np.copy(x)
                x_ij[i] += h #sumamos h en la i-esima entrada
                x_ij[j] += h #sumamos h en la j-esima entrada
                
                #vector x + he_i
                x_i= np.copy(x)
                x_i[i] += h #sumamos h en la i-esima entrada

                #vector x + he_j
                x_j= np.copy(x) #sumamos h en la i-esima entrada
                x_j[j] += h

                hessian[i, j] = (f(x_ij)-f(x_i)-f(x_j)+f(x)) / (h**2)
                #la matriz es simétrica:
                hessian[j, i] = hessian[i, j]
    return hessian


In [152]:
def errorRelativo_grad(fncf, gradf, n, nt, h):
    '''
    Imprime estadísticas sobre el error relativo entre el gradiente analítico
    de una función y el cálculado con autograd.
    fncf  - Función f
    gradf - Implementación del gradiente analítico de la función f
    n     - Dimensión de la variable n
    nt    - Número de puntos aleatorios en donde se comparan los gradientes 
    h     - tamaño de paso
    '''
    ve = np.zeros(nt)  # Arreglo para almacenar errores relativos
    
    g_f = grad(fncf)  # Función gradiente generada con autograd
    for it in range(nt):
        x0 = np.random.randn(n)
        g0 = gradf(fncf, x0, h)
        ga = g_f(x0)
        ve[it] = np.linalg.norm(g0 - ga) / np.linalg.norm(ga)
    
    min_error = np.min(ve)
    max_error = np.max(ve)
    mean_error = np.mean(ve)
    print('Min: %.2e   Media: %.2e    Max: %.2e' % (min_error, mean_error, max_error))
    



In [165]:
h_list=[10e-5,10e-6,10e-7,10e-8]

**Función cuadrática con A1**

In [176]:
n_list=[10,100,1000]

for n in n_list:
    print("Para n=", n)
    print("---------------------------------------------")
    A=genera_A1(n)
    b=np.ones(n)
    cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
    for h in h_list:
        print("h= ", h)
        errorRelativo_grad(cuadratica, forward_difference, n,100,h)
        print("\n")

Para n= 10
---------------------------------------------
h=  0.0001
Min: 2.92e-05   Media: 5.52e-05    Max: 1.08e-04


h=  1e-05
Min: 2.17e-06   Media: 5.34e-06    Max: 1.42e-05


h=  1e-06
Min: 2.57e-07   Media: 5.37e-07    Max: 1.03e-06


h=  1e-07
Min: 2.05e-08   Media: 5.51e-08    Max: 1.21e-07


Para n= 100
---------------------------------------------
h=  0.0001
Min: 4.23e-05   Media: 5.06e-05    Max: 6.02e-05


h=  1e-05
Min: 4.24e-06   Media: 5.07e-06    Max: 5.90e-06


h=  1e-06
Min: 3.90e-07   Media: 4.96e-07    Max: 5.97e-07


h=  1e-07
Min: 2.98e-08   Media: 1.11e-07    Max: 3.63e-07


Para n= 1000
---------------------------------------------
h=  0.0001
Min: 4.81e-05   Media: 5.01e-05    Max: 5.24e-05


h=  1e-05
Min: 4.64e-06   Media: 5.00e-06    Max: 5.27e-06


h=  1e-06
Min: 4.06e-08   Media: 4.97e-07    Max: 1.26e-06


h=  1e-07
Min: 3.06e-07   Media: 2.59e-06    Max: 6.93e-06




* Para h=10
El mejor h es 10-7
* Para h=100
El mejor h es 10-7
* Para h=1000

**Función cuadrática con A2**

In [177]:
n_list=[10,100,1000]

for n in n_list:
    print("Para n=", n)
    print("---------------------------------------------")
    A=genera_A2(n)
    b=np.ones(n)
    cuadratica = lambda x: 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)
    for h in h_list:
        print("h= ", h)
        errorRelativo_grad(cuadratica, forward_difference, n,100,h)
        print("\n")

Para n= 10
---------------------------------------------
h=  0.0001
Min: 1.19e-05   Media: 3.42e-05    Max: 1.25e-04


h=  1e-05
Min: 1.20e-06   Media: 3.41e-06    Max: 1.35e-05


h=  1e-06
Min: 1.31e-07   Media: 3.86e-07    Max: 1.68e-06


h=  1e-07
Min: 7.28e-09   Media: 3.38e-08    Max: 1.16e-07


Para n= 100
---------------------------------------------
h=  0.0001
Min: 1.97e-05   Media: 2.70e-05    Max: 3.88e-05


h=  1e-05
Min: 2.00e-06   Media: 2.74e-06    Max: 3.59e-06


h=  1e-06
Min: 1.81e-07   Media: 2.73e-07    Max: 3.82e-07


h=  1e-07
Min: 1.82e-08   Media: 4.68e-08    Max: 1.05e-07


Para n= 1000
---------------------------------------------
h=  0.0001
Min: 2.39e-05   Media: 2.68e-05    Max: 3.00e-05


h=  1e-05
Min: 2.41e-06   Media: 2.66e-06    Max: 2.94e-06


h=  1e-06
Min: 2.18e-07   Media: 2.73e-07    Max: 3.20e-07


h=  1e-07
Min: 2.39e-07   Media: 3.62e-07    Max: 7.31e-07




* Para h=10
* Para h=100
* Para h=1000

**Función de Beale**

In [164]:
x0=(2,3)
n=len(x0)
print("Para n=", n)
for h in h_list:
    print("h= ", h)
    errorRelativo_grad(fn.Beale, forward_difference, n,100,h)
    print("\n")

Para n= 2
h=  0.0001
Min: 2.04e-05   Media: 9.61e-05    Max: 1.25e-03


h=  1e-05
Min: 8.07e-07   Media: 1.25e-05    Max: 1.88e-04


h=  1e-06
Min: 1.18e-07   Media: 1.45e-06    Max: 5.16e-05


h=  1e-07
Min: 5.23e-09   Media: 1.03e-07    Max: 1.44e-06




**Función de Himmelblau** 

In [167]:
x0=(2,4)
n=len(x0)
print("Para n=", n)
for h in h_list:
    print("h= ", h)
    errorRelativo_grad(fn.Himmelblau, forward_difference, n,100,h)
    print("\n")

Para n= 2
h=  0.0001
Min: 6.08e-06   Media: 7.75e-05    Max: 6.05e-04


h=  1e-05
Min: 5.04e-07   Media: 8.22e-06    Max: 7.26e-05


h=  1e-06
Min: 5.64e-08   Media: 8.17e-07    Max: 6.67e-06


h=  1e-07
Min: 1.01e-08   Media: 6.80e-08    Max: 4.31e-07




**Función de Rosenbrock**

In [172]:
x0_list=[(-1.2,1.0),(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0),(-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0,-1.2,1.0)]

for x0 in x0_list:
    n=len(x0)
    print("Para n=", n)
    print("---------------------------------------------")
    for h in h_list:
        print("h= ", h)
        errorRelativo_grad(fn.Rosenbrock, forward_difference, n,100,h)
        print("\n")

Para n= 2
---------------------------------------------
h=  0.0001
Min: 3.15e-05   Media: 2.14e-04    Max: 5.45e-03


h=  1e-05
Min: 2.31e-06   Media: 1.38e-05    Max: 1.47e-04


h=  1e-06
Min: 2.66e-07   Media: 2.54e-06    Max: 4.18e-05


h=  1e-07
Min: 4.72e-08   Media: 1.96e-07    Max: 1.79e-06


Para n= 20
---------------------------------------------
h=  0.0001
Min: 4.77e-05   Media: 7.52e-05    Max: 1.12e-04


h=  1e-05
Min: 4.55e-06   Media: 7.56e-06    Max: 1.38e-05


h=  1e-06
Min: 4.76e-07   Media: 7.35e-07    Max: 1.28e-06


h=  1e-07
Min: 4.21e-08   Media: 7.50e-08    Max: 1.37e-07


Para n= 40
---------------------------------------------
h=  0.0001
Min: 4.30e-05   Media: 6.94e-05    Max: 1.09e-04


h=  1e-05
Min: 3.81e-06   Media: 6.96e-06    Max: 1.02e-05


h=  1e-06
Min: 3.90e-07   Media: 7.09e-07    Max: 1.02e-06


h=  1e-07
Min: 4.09e-08   Media: 6.88e-08    Max: 1.06e-07




$\color{purple}{Ejercicio \space 3}$ <br>

Paper:

    Nonlinear conjugate gradient for smooth convex function

Integrantes:
* Ramon Orlando Ruíz Olais, MCC CIMAT
* Yesenia Sarahi Garcia Gonzalez, MCC CIMAT

